In [3]:
# Import some basic libraries and functions for this tutorial.
import numpy as np
import os

from pydrake.common import temp_directory
from pydrake.geometry import StartMeshcat
from pydrake.math import RigidTransform, RollPitchYaw, RotationMatrix
from pydrake.multibody.parsing import Parser
from pydrake.multibody.plant import AddMultibodyPlantSceneGraph
from pydrake.systems.analysis import Simulator
from pydrake.systems.framework import DiagramBuilder
from pydrake.visualization import AddDefaultVisualization, ModelVisualizer

print("Check!")

Check!


In [4]:
from pydrake.all import (
    AddMultibodyPlantSceneGraph, DiagramBuilder, 
    FindResourceOrThrow, GenerateHtml, InverseDynamicsController, 
    MultibodyPlant, Parser, Simulator)
from pydrake.multibody.jupyter_widgets import MakeJointSlidersThatPublishOnCallback
import pydrake
from pydrake import geometry
from pydrake.math import RigidTransform, RollPitchYaw, RotationMatrix 
from pydrake.solvers import MathematicalProgram, Solve
from pydrake.systems.jupyter_widgets import PoseSliders, WidgetSystem
from ipywidgets import ToggleButton, ToggleButtons
from functools import partial
from pydrake.all import (
    JointIndex, PiecewisePolynomial, JacobianWrtVariable,
    eq, ge,  AutoDiffXd, SnoptSolver, IpoptSolver,  
    AddUnitQuaternionConstraintOnPlant, PositionConstraint, OrientationConstraint
    )
print("Check!")

Check!


# 1 Setting up the scene
#### some notes:

from pydrake.all import (
    autoDiffToGradientMatrix, initializeAutoDiffGivenGradientMatrix, autoDiffToValueMatrix, autoDiffToGradientMatrix)

**These imports seem to be renovated, should have equivalent. Do check!** 

In [5]:
h = 1e-3
builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=h)
parser = Parser(plant)

In [6]:
dir1 = "/Users/xiao/0_codes/ICBM_drake/models/objects"
dir2 = "/Users/xiao/0_codes/ICBM_drake/models/ycb/sdf/"
table_file = os.path.join(dir1, "table_top.sdf")
panda_file = "/Users/xiao/0_codes/ICBM_drake/models/franka_description/urdf/panda_arm_hand.urdf"
box_file = os.path.join(dir2, "003_cracker_box.sdf")

# scene_dir = ("/Users/xiao/0_codes/ICBM_drake/scenes/scene1")
# These add multiple models and returns a list
table_modelInstance = parser.AddModels(table_file)[0]  # this return pydrake.multibody.tree.ModelInstanceIndex
panda_modelInstance = parser.AddModels(panda_file)[0]
box_modelInstance = parser.AddModels(box_file)[0]

INFO:drake:PackageMap: Downloading https://github.com/RobotLocomotion/models/archive/90397c33cab9b7234d94c1018f2755bb9989c5a8.tar.gz


In [7]:
print(table_modelInstance)
print(type(table_modelInstance))

ModelInstanceIndex(2)
<class 'pydrake.multibody.tree.ModelInstanceIndex'>


In [8]:
# get table-top frame in the world, here the dummy is for getting familiar with Drake
table_top_frame = plant.GetFrameByName("table_top_center")
robot_base = plant.GetFrameByName("panda_link0")

# move the table for a bit
X_W_table = RigidTransform(
  R=RotationMatrix([
    [1.0, 0.0, 0.0],
    [0.0, 1.0, 0.0],
    [0.0, 0.0, 1.0],]),
  p=[0.0, 0.0, 0.0]
) # or similar: X_W_table = RigidTransform(RotationMatrix.Identity(), [0., -0.4, 0.])

plant.WeldFrames(plant.world_frame(), table_top_frame, X_W_table) # 2nd arg relative to 1st arg transformed by X_W_table 

# fix panda to table top.
plant.WeldFrames(table_top_frame, robot_base, RigidTransform(RotationMatrix.Identity(), [0., -0.4, 0.]))


# plant.WeldFrames(X_W_table, table_top_frame)
print(f"table_top_frame is {table_top_frame}")

""" These will be depracated on 2023-12-01
Deprecated:
    Use parser.AddModels() instead. To port the 2-argument form,
    rename models using parser.SetAutoRenaming() and
    plant.RenameModelInstance(). See PR #19978 for more details. This
    will be removed from Drake on or after 2023-12-01.
  parser.AddModelFromFile(table_file, "table") """
# # using add model from file 
# parser.AddModelFromFile(table_file, "table")
# parser.AddModelFromFile(box_file, "box")

"""
# This is explicitly fixing the table to world frame. What happens if it is not
# explicitly fixed here but "not publishing to update its state" ?? See later sections.  

# fix the centre of table to world
table_body_frame = plant.GetFrameByName("table_top_center")
plant.WeldFrames(plant.world_frame(), table_body_frame) 
"""

# finalize the plant
plant.Finalize()
plant.set_name("table_with_box")

# now build the diagram
diagram = builder.Build()

table_top_frame is <FixedOffsetFrame name='table_top_center' index=3 model_instance=2>


## 1 On setting fixed body pose in world
Use $plant.WeldFrames(parentFrame,\;childFrame, \;Transformation)$ to fix objects in the scene. For fixed objects, this needs to be done before $plant.Finalize()$
### 1.1 for getting frames:
plant.world_frame(), 
plant.GetFrameByName()
### 1.2 using drake transformations:
The transformation X should be of type "*pydrake.math.RigidTransform*"
e.g.:
X_W_table = RigidTransform(
  R=RotationMatrix([
    [1.0, 0.0, 0.0],
    [0.0, 1.0, 0.0],
    [0.0, 0.0, 1.0],]),
  p=[0.0, 0.0, 0.0],)

In [9]:
# get free body box and its id in the plant
box_body = plant.GetBodyByName("base_link_cracker") # get the body instance
box_body_id = plant.GetBodyFrameIdIfExists(box_body.index()) # get the body's id in the plant

table_top_body = plant.GetBodyByName("table_top_link") # since only one from each pool
table_top_frame_id = plant.GetBodyFrameIdIfExists(table_top_body.index())

panda_base = plant.GetBodyByName("panda_link0")
panda_base_frame_id = plant.GetBodyFrameIdIfExists(panda_base.index())

panda_hand = plant.GetBodyByName("panda_hand")
panda_hand_frame_id = plant.GetBodyFrameIdIfExists(panda_hand.index())

print(f"box_body_frame is {box_body} \n")
print(f"box_body_id is {box_body_id}")

print(type(box_body))

print(type(box_body_id))

# this 
print(box_body.body_frame()) # this get the body frame from the body instance

box_body_frame is <RigidBody name='base_link_cracker' index=14 model_instance=4> 

box_body_id is <FrameId value=268>
<class 'pydrake.multibody.tree.RigidBody'>
<class 'pydrake.geometry.FrameId'>
<BodyFrame name='base_link_cracker' index=27 model_instance=4>


In [10]:
panda_base.body_frame()
panda_hand.body_frame()

<BodyFrame name='panda_hand' index=13 model_instance=3>

In [11]:
# create the default diagram's context 
context = diagram.CreateDefaultContext()

# get the mutable context for updates
mutable_context = plant.GetMyContextFromRoot(context) # seems equivalent to .GetMyMutableContextFromRoot()

# # move the table for a bit
# X_W_table = RigidTransform(RotationMatrix.Identity(), [0., -0.4, 0.])

# plant.SetFreeBodyPose(mutable_context, plant.GetBodyByName("table_top_link"), X_W_table)

# move the box a bit relative to the table-top
X_table_box = RigidTransform(RollPitchYaw(np.asarray([0, 0, 0]) * np.pi / 180), p=[0.1, -0.2, 0.])
X_W_table_dummy = table_top_frame.CalcPoseInWorld(mutable_context)
X_W_box = X_W_table_dummy.multiply(X_table_box)
plant.SetFreeBodyPose(mutable_context, box_body, X_W_box)

In [12]:
meshcat = StartMeshcat()
# not yet publish any context for visualization

INFO:drake:Meshcat listening for connections at http://localhost:7001


In [13]:
# Need this because a==b returns True even if a = AutoDiffXd(1, [1, 2]), b= AutoDiffXd(2, [3, 4])
# That's the behavior of AutoDiffXd in C++, also.
def autoDiffArrayEqual(a,b):
    return np.array_equal(a, b) and np.array_equal(ExtractGradient(a), ExtractGradient(b))

In [14]:
from pydrake.all import (AutoDiffXd, ExtractGradient)
import numpy as np
a = AutoDiffXd(1, [1, 2])
b = AutoDiffXd(2, [3, 4]) 

In [15]:
N = 100

In [16]:
"""
one default context of the diagram per time step and each corresponds to one plant context? 
It this needed?
To use SceneGraph, the plant context need to be derived from diagram
"""

# Create one context per timestep to maximize cache hits
context_list = [diagram.CreateDefaultContext() for i in range(N)] 
plant_context_list = [plant.GetMyContextFromRoot(context_list[i]) for i in range(N)]

#or just:
# plant_context_list = [plant.CreateDefaultContext() for i in range(N)]

In [17]:
# convert to AutoDiff compatible
ad_diagram = diagram.ToAutoDiffXd()
ad_plant = ad_diagram.GetSubsystemByName("table_with_box")

In [18]:
# create new AutoDiff context to maximize cache hits
""" To use SceneGraph, the plant context need to be derived from diagram """
ad_context_list = [ad_diagram.CreateDefaultContext() for i in range(N)]
ad_plant_context_list = [ad_plant.GetMyContextFromRoot(ad_context_list[i]) for i in range(N)]

In [19]:
# the context here is the very first one 
""" Notices here the box is an list of pydrake.multibody.tree.ModelInstanceIndex after parser.AddModels(...) """
box_mass = sum(plant.get_body(index).get_mass(context) for index in plant.GetBodyIndices(box_modelInstance))
panda_mass = sum(plant.get_body(index).get_mass(context) for index in plant.GetBodyIndices(panda_modelInstance))
gravity = plant.gravity_field().gravity_vector()

print(f"The box's mass is: {box_mass}\n")
print(f"Total mass of panda is: {panda_mass}\n")
print(f"The gravity vector is: {gravity}\n")

The box's mass is: 0.411

Total mass of panda is: 18.92

The gravity vector is: [ 0.    0.   -9.81]



# Section 2: setting up the decVars and prog()

<div class="alert alert-block alert-danger">
<b>Need to think carefully here!:</b> The decision variable should include also $\gamma_k$, the slack variables, one for each contact; the tangential contact forces $\lambda^+$ and $\lambda^-$ are enough in 2D case, the difference of which determines the magnitude of the resultant tangential force at the contact. In 3D cases, these $\lambda$s and also the weights for each basis vector of the approximation of the friction cone. The number of the weights depends on the discretization of the cone. Saying, approx by hexagon, then we have 6 more decision variables, which is to produce the direction of one of the tangential friction (the other is exactly the opposite, so 6 is enough). The direction of the tangential friction is $\hat{f}_t = w_1 \hat{b}_1 + w_2 \hat{b}_2 + w_3 \hat{b}_3 + w_4 \hat{b}_4 + w_5 \hat{b}_5 + w_6 \hat{b}_6$ expressed in the frame at the contact point $\{C\}$. Therefore, in this case with panda + a box, assuming 5 contact points (1 hand-box, 4 box-table), nDecVar = 7$\times$2 + 6$\times$2 + 1$\times$5 + 3$\times$5 + 6$\times$5 = arm_dof + box_dof + slackVar + $\lambda$s + coneWeights = 76. Damn!!
    The $\alpha$ and $\beta$ are used as the trick to improve convergence in solving with SQP (e.g. SNOPT) in Section 3.2 of the paper. Let's try solving it first using interior-point methods.
</div>

<div class="alert alert-block alert-warning">
    <b>Here I set the convention of the ordering of the decision variables:</b> 
</div>

The decision variable set will be ordered as:
***
__[q_arm, v_arm, q_objects, v_objects, c_params, coneBasis]__, where within c_params, each sub-set [$\lambda_{Nk}, \lambda^+_k, \lambda^-_k, \gamma_k$] corresponds to one contact point in 2D. The contact type are ordered like this: hand-obj, obj-obj, obj-env.


In [43]:
7*2+6*2+5+15+30

76

In [33]:
T = 0.5
prog = MathematicalProgram()

""" # maybe skip the h_var for now, this means the time step is fixed.`;
# bind time step 
h_var = prog.NewContinuousVariables(N-1, "h")
"""

# create the decVar vector: arm_dof + box_dof + $\lambda$s + slackVar +  + coneWeights 
# KEEP THE ORDERING CONSISTANT!
nq_box = plant.num_positions(box_modelInstance)
nv_box = plant.num_velocities(box_modelInstance)
nq_arm = plant.num_positions(panda_modelInstance)
nv_arm = plant.num_velocities(panda_modelInstance)

# 1 hand-obj, 4 obj-env
n_contact = 5

# 7 + 2, 2 are two 'fixed' finger joints
assert nq_arm == 9, "# of joints does not match, something is wrong!"

# the return type of prog.NewContinuousVariables() is numpy.ndarray
# decVars: for arm and one object
q_box_vars = prog.NewContinuousVariables(nq_box, N, "q_box") # [row, col]: [nq_o, N]
v_box_vars = prog.NewContinuousVariables(nv_box, N, "v_box")
q_arm_vars = prog.NewContinuousVariables(nq_arm-2, N, "q_arm")
v_arm_vars = prog.NewContinuousVariables(nv_arm-2, N, "q_arm")

# decVars: c_params, [N, +, -, alpha, beta]
c_params_vars = prog.NewContinuousVariables(5*n_contact, N, "c_params")

# decVars: cone basis, assuming a hexagon approximation
cone_vars = prog.NewContinuousVariables(6*n_contact, N, "coneBasis")

print(f"h_var has type: {type(h_var)}, and shape: {h_var.shape}\n")
print(f"h_var[0] has type: {type(h_var[0])}\n")

h_var has type: <class 'numpy.ndarray'>, and shape: (99,)

h_var[0] has type: <class 'pydrake.symbolic.Variable'>



' This should not be here. \n    constraints on acceleration should come from the dynamics. '

In [32]:
"""
# DO NOT bother with variable time step just yet 
# constraints on time
prog.AddBoundingBoxConstraint([0.5*T/(N-1)]*(N-1), [2.0*T/(N-1)]*(N-1), h_var)
prog.AddLinearConstraint(sum(h_var) >= 0.9*T)
prog.AddLinearConstraint(sum(h_var) <= 1.5*T)
prog.SetInitialGuess(h_var, [T/(N-1)]*(N-1))
"""

' # DO NOT bother with variable time step just yet \n# constraints on time\nprog.AddBoundingBoxConstraint([0.5*T/(N-1)]*(N-1), [2.0*T/(N-1)]*(N-1), h_var)\nprog.AddLinearConstraint(sum(h_var) >= 0.9*T)\nprog.AddLinearConstraint(sum(h_var) <= 1.5*T)\nprog.SetInitialGuess(h_var, [T/(N-1)]*(N-1))\n'

In [23]:
# boundary constraints
q0_box = plant.GetPositions(mutable_context, box_modelInstance)
v0_box = plant.GetVelocities(mutable_context, box_modelInstance)
q0_arm = plant.GetPositions(mutable_context, panda_modelInstance)[:7] # excluding the fingers
v0_arm = plant.GetVelocities(mutable_context, panda_modelInstance)[:7] 
                             

print(f"The box initial q is: {q0_box} \n")
print(f"The box initial v is: {v0_box} \n")
print(f"The panda initial q is: {q0_arm} \n")
print(f"The panda initial v is: {v0_arm} \n")

The box initial q is: [ 1.   0.   0.   0.   0.1 -0.2  0. ] 

The box initial v is: [0. 0. 0. 0. 0. 0.] 

The panda initial q is: [0. 0. 0. 0. 0. 0. 0.] 

The panda initial v is: [0. 0. 0. 0. 0. 0. 0.] 



In [24]:
# set init v and vdot if needed
vdot0_box = np.array([0., 0., 0., 0., 0., -9.81]) # not sure if this is needed
v0_box = np.array([0., 0., 0., 0., 0., 0.]) # twist [wx, wy, wz, vx, vy, vz]

# set box constraints on the initial states
prog.AddBoundingBoxConstraint(q0_box, q0_box, q_box_vars[:, 0]) # at t = 0
prog.AddBoundingBoxConstraint(v0_box, v0_box, v_box_vars[:, 0]) # at t = 0
prog.AddBoundingBoxConstraint(q0_arm, q0_arm, q_arm_vars[:, 0]) 
prog.AddBoundingBoxConstraint(v0_arm, v0_arm, v_arm_vars[:, 0]) 

In [26]:
""" # this is adds unit quaternion constraints to all quaternion joints in the plant

# add unit quaternion constraint for each time point
for i in range(N):
    # This constraint is added to the generalized positions not just the rotation part?? Double check this in C++
    # yes, to generalized positions
    AddUnitQuaternionConstraintOnPlant(plant, q_o_vars[:, i], prog) # The decision variables for the generalized position of the plant.
"""

# what I need is unit quaternion constraints on the object at all time points
from pydrake.multibody.inverse_kinematics import UnitQuaternionConstraint
for i in range(N):
    prog.AddConstraint(UnitQuaternionConstraint(), q_box_vars[3:, i]) # double check if positions are arranged [x, y, z, w, i, j, k]

In [27]:
tester = np.arange(7)
print(tester)
print(tester[4:])
print(tester[3:])

[0 1 2 3 4 5 6]
[4 5 6]
[3 4 5 6]


In [34]:
q_box_vars.shape

(7, 100)

In [35]:
plant.num_positions() # after fixing the table to world, this drops from 14 to 7

16

In [42]:
floating_bodies = plant.GetFloatingBaseBodies() # this returns a python set
print(f"Total: {len(floating_bodies)} floating bodies in the world")
print(n_contact)
frame_hand = ad_plant.GetFrameByName("panda_link8")
frame_hand

Total: 1 floating bodies in the world
5


<BodyFrame_[AutoDiffXd] name='panda_link8' index=12 model_instance=3>

# 3 Adding constraints on the decVars


<div class="alert alert-block alert-warning">
<b>NOTE:</b> The selected point in its corresponding frame should be passed in. Before enforcing the manipulator equation constraint, all the contact points are made available by contact selector, pose estimation and assumptions. For sliding contact: for one contact, a contact point in {hand}, also the same point in {box}; these two should be resolved in {contact-point} for relative tangential velocity. 
</div>

$\underline{\lambda} = (\lambda_1, \lambda_2, \lambda_3, \lambda_4, \lambda_5, \lambda_6)$, one for each basis. 

<div class="alert alert-block alert-warning">
<b>To do:</b> Need to figure out how to retrieve points from geometry. The point is expressed in {model} by default using signed_distance_pairs = query_object.ComputeSignedDistancePairwiseClosestPoints() in Drake's class drake::geometry::QueryObject<T>. Since I am not using the part of pipeline, need to find a way to express the point in frames. Let's just assume the selected points are in the {model} and passed into def manipulator_equation_cst(). 
</div>

In [52]:
# the manipulator equation constraint
def manipulator_equation_cst(decVars, context_index, p_contacts):
    """
    this function implements the system dynamics in implicit form, as non-linear eq constraint, using implicit Euler integration: 
    
    0 = [ Buₙ₊₁ + ∑ᵢ (Jᵢ_WBᵀ(qₙ₊₁)ᵀ * Fᵢ_AB_W(λᵢ,ₙ₊₁)) + tau_g(qₙ₊₁) - C(qₙ₊₁, Vₙ₊₁) ] * dt - M(qₙ₊₁) * (Vₙ₊₁ - Vₙ)
    
    decVars: [vₙ, qₙ₊₁, vₙ₊₁, uₙ₊₁, λₙ₊₁], which is a subset of the decVars of the overall system. 
             lambda contains N, lambda1, lambda2, lambda3, lambda4, lambda5, lambda6
             The format of the input to the Eval() function is a vector containing: {vₙ, qₙ₊₁, vₙ₊₁, uₙ₊₁, λₙ₊₁},
             where λₙ₊₁ is a concatenation of lambdas for all contacts

    context_index: one context of a plant for each time point.
    p_contacts: [nc, 3], 
                the 3d coordinates of the points in {model}, which is relative to the object model; 
                the ordering priority is h-o > o-o > o-env
    """
    # unpack decVars
    v_n, q_next, v_next, u_next, lambdas = np.split(decVars, [nq_o, 
                                                                  nq_o+nq_v, 
                                                                  nq_o+nq_v*2, 
                                                                  nq_o+nq_v*2+n_u, 
                                                                  nq_o+nq_v*2+n_u])
    
    # update context position and velocity
    if isinstance(decVars[0], AutoDiffXd):
        if not autoDiffArrayEqual(q_next, ad_plant.GetPositions(ad_plant_context_list[context_index], box)):
            ad_plant.SetPositions(ad_plant_context_list[context_index], box, q_next)
        if not autoDiffArrayEqual(v_next, ad_plant.GetVelocities(ad_plant_context_list[context_index], box)):
            ad_plant.SetVelocities(ad_plant_context_list[context_index], box, v_next)
        
        # this is needed to commodate ad and non-ad plant, to avoid coding it twice
        eval_plant = ad_plant
        eval_plant_context = ad_plant_context_list[context_index]
            
    else:
        if not np.array_equal(q, plant.GetPositions(plant_context_list[context_index], box)):
            plant.SetPositions(plant_context_list[context_index], box, q_next)
        if not np.array_equal(v, plant.GetVelocities(plant_context_list[context_index], cracker_box)):
            plant.SetVelocities(plant_context_list[context_index], box, v_next)

        eval_plant = plant
        eval_plant_context = plant_context_list[context_index]

    # this selects the actuated dofs, which by passes the unactuated generalized v of objects 
    B = plant.MakeActuationMatrix()
    
    # control with selection matrix
    y = B @ u_next
    g_term = eval_plant.CalcGravityGeneralizedForces(eval_plant_context)

    # gravity
    y += g_term
    c_term = eval_plant.CalcBiasTerm(eval_plant_context)

    # velocity product (or bias term) 
    y -= c_term
    M = eval_plant.CalcMassMatrixViaInverseDynamics(eval_plant_context)

    # now the contact, which has two part: contact jacobian and the contact wrench
    # first off, the contact Jacobian 
    """ Jv_V_WCa reads the spatial velocity(V) Jacobian of contact point a (Ca) expressed in {W} w.r.t the generalize v 
        the point in body frame is required by SELECTION """
    # get frame of the hand(link-8), and a point on the hand in this frame 
    frame_hand = eval_plant.GetFrameByName("panda_link8")

    # a point relative to frame_hand, here it is chosen arbitrarily. Consort to the real model for this point or from contact selector.
    p_in_hand = np.array([[0.1],[0.1],[0.1]])

    """
    These two jacobians are required to map contact forces to each 'system'
    By Newton's 3rd Law, one for hand; then the opposite to the object. Equal and opposite.
    The Jacobians here has sparsity structure, zeros for object in the part mapping to 
    the arm, and likewise for the part mapping to the object.
    """
    # Jacobian of a point in {hand} evaluated in {W}
    Jv_V_W_Chand = eval_plant.CalcJacobianSpatialVelocity(eval_plant_context, 
                                                      JacobianWrtVariable.kV, 
                                                      frame_hand, 
                                                      p_in_hand, 
                                                      eval_plant.world_frame(), 
                                                      eval_plant.world_frame())
    # box frame
    frame_box = eval_plant.GetFrameByName("base_link_cracker")
    # the contact point in box frame, this is an arbitray choice
    p_in_box = np.array([[0.1],[0.1],[0.1]])
    
    # Jacobian of a point in {box}, this Jacobian is just the Identity Matrix 
    # 6-spatial-velo = J @ 6-generalized-dof.
    Jv_V_W_Cbox = eval_plant.CalcJacobianSpatialVelocity(eval_plant_context, 
                                                      JacobianWrtVariable.kV, 
                                                      frame_box, 
                                                      p_in_box, 
                                                      eval_plant.world_frame(), 
                                                      eval_plant.world_frame())

    # Compute the wrench in {W} using the decVar
    # 1. get the contact frame (establish its convention)
    # 2. express the contact normal in {contact_point}
    # 3. express tangential direction in {contact_point} using the cone basis,
    #    i.e. the positive span produces a tangential vector
    # 4. express the resultant force (fn + ft) in {W}
    # The contact wrench in {W} is [0, 0, 0, fx, fy, fz], which is resolved from fc = fn + ft

    # this reads: the twist of {hand} in {box} expressed in {W}  
    V_box_hand_W = frame_hand.CalcSpatialVelocity(eval_plant, frame_box, eval_plant.world_frame())
    # Now need to get the p_{model}
    pydrake.geometry.SceneGraphInspector.GetPoseInFrame() Find equivalent or manually compute it!!
    """ Continue on:
    line 211 in sliding_friction_complementarity_constraint.cc
    
    // We use Bg to mean the geometry frame attached to body B, and Bb to mean
    // the body frame of body B.
    const Vector3<AutoDiffXd>& p_BgCb = signed_distance_pair.p_BCb;
    const Vector3<AutoDiffXd> p_BbCb =
        inspector.GetPoseInFrame(signed_distance_pair.id_B).template cast<AutoDiffXd>() * p_BgCb;
    """
    

    
    """ continue on reading manipulator_equation_constraint_text.cc 
    There must be the part to resolve the fc = fn + ft into {W} in the SceneGraph() or inspector() or whatever that
    I did not see in the doEval() in contact_wrench_evaluator.cc """

    # compute M(q)
    eval_plant.CalcMassMatrixViaInverseDynamics(eval_plant_context)

    """ 
        AutoDiffVecXd F_AB_W;
    it->second.contact_wrench_evaluator->Eval(
        it->second.contact_wrench_evaluator->ComposeVariableValues(*context_,
                                                                   lambda),
        &F_AB_W);

    // By definition, F_AB_W is the contact wrench applied to id_B from id_A,
    // at the contact point. By Newton's third law, the contact wrench applied
    // to id_A from id_B at the contact point is -F_AB_W.
    *y += Jv_V_WCa.transpose() * -F_AB_W + Jv_V_WCb.transpose() * F_AB_W;
  }
    """
    
    """
    or
    Jv_v_WCa = eval_plant.CalcJacobianTranslationalVelocity()
    Jv_v_WCb = eval_plant.CalcJacobianTranslationalVelocity()
    
    these are needed to compute the relative velocity at contact point.

    Make sure I understand all the jacobians tools in drake!
    """
    
    # initialize the ee_term which is the same dim as the arm dof
    ee_wrench = np.zeros(nv_o)
    
    # need to compute the contact Jacobian based on selected points (or project onto surface) 
    # rather than using SceneGraph inspector
    y += 

AttributeError: module 'numpy' has no attribute 'identify'

In [41]:
t = np.arange(10)
print(t)

a = np.split(t, [2, 4, 6])
print(a)

[0 1 2 3 4 5 6 7 8 9]
[array([0, 1]), array([2, 3]), array([4, 5]), array([6, 7, 8, 9])]


In [42]:
plant.num_actuated_dofs()

0

In [50]:
isinstance(q_o_vars[0, 0], AutoDiffXd)

False

In [51]:
type(q_o_vars[0][0])

pydrake.symbolic.Variable

In [53]:
np.zeros((5))

array([0., 0., 0., 0., 0.])

In [54]:
np.zeros(5)

array([0., 0., 0., 0., 0.])